# CFEM - Itabira

## 1. Coleta de Dados

### 1.1 Conexão

In [166]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import json
from bs4 import BeautifulSoup
import os

driver = webdriver.Chrome()
url_paginaInicial = "https://servicositabira.govbr.cloud/pronimtb/"
url_despesas_fonteDeRecurso = "https://servicositabira.govbr.cloud/pronimtb/index.asp?acao=3&item=6"

driver.get(url_despesas_fonteDeRecurso) # Pagina Inicial > Despesas > Fonte de Recurso

# Esperar até que o DOM esteja carregado
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


<selenium.webdriver.remote.webelement.WebElement (session="510abd50e0733730b8e1ac5900950682", element="f.4DC2686E5BA0CFCB0C6A6E38F4777E58.d.54CD0BBA71F83CB8F14852DF5BDE7411.e.247")>

### 1.2 Filtros

In [167]:
# Data inicial
data_inicial = driver.find_element(By.ID, "txtDataInicial")
data_inicial.clear()
data_inicial.send_keys("01012024")

# Data final 
data_final = driver.find_element(By.ID, "txtDataFinal")
data_final.clear()
data_final.send_keys("31012024")


# Unidade Gestora
select_element = driver.find_element(By.ID, 'cmbUnidadeGestora')
select = Select(select_element)
select.select_by_value('0')  # Selecionar a opção com valor '0'

# Clicar no botão "Gerar"
element = driver.find_element(By.ID, "confirma")
element.click()

### 1.3 Função ler_tabela()

In [168]:
import json
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def ler_tabela(nome_dados):
    global dados_tabelas

    html_table = wait.until(EC.presence_of_element_located((By.ID, 'tbTabela')))
    
    # Passar o objeto html_table para BeautifulSoup
    soup = BeautifulSoup(html_table.get_attribute('outerHTML'), 'html.parser')
    
    # Encontra o elemento da tabela
    table = soup.find('table')
    
    # Inicializa dicionário vazio
    table_data = {}
    
    # Extrai headers
    header_row = table.find_all('tr')[1]  # Assumindo a segunda linha como header
    headers = header_row.find_all('th')
    headers_text = [header.text.strip().replace('.', '') for header in headers]  # Remove pontos
    
    # Extrai dados da tabela
    for row in table.find_all('tr')[2:]:  # Pula headers
        cells = row.find_all('td')
        item = cells[0].get_text().strip().replace('.', '')  # Remove pontos
        # Remove todos os pontos
        cells_text = [cell.get_text().strip().replace('.', '') for cell in cells[1:]]
        data = {
            headers_text[i+1]: cells_text[i] for i in range(len(cells_text))
        }
        table_data[item] = data

    # Adiciona os dados ao dicionário global
    if nome_dados not in dados_tabelas:
        dados_tabelas[nome_dados] = table_data
    else:
        dados_tabelas[nome_dados].update(table_data)
    
    # Converte dados para JSON identado
    json_data = json.dumps(table_data, indent=4, ensure_ascii=False)
    
    # Lidando com diretório
    directory = 'json_output'
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{nome_dados}.json')
    
    # Salvando JSON em arquivo
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json_file.write(json_data)
    print(f'Dados salvos em: {file_path}')


### 1.4 Seleção de Recursos

1.4.1 CFEM

In [169]:
# Selecionando CFEM na Tabela # Pagina Inicial > Despesas > Fonte de Recurso > CFEM
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
try:
    # Localize o link pelo texto
    link_cfem = driver.find_element(By.LINK_TEXT, "Transf União Referente à Compensação Financeira de Recursos Minerais")
    
    # Clique no link
    link_cfem.click()

except TimeoutException:
    print("Ocorreu um timeout ao esperar por um elemento.")
except NoSuchElementException:
    print("Não foi possível encontrar um elemento.")


1.4.2 Detalhamento da fonte de despesa (CFEM Ordinário)

In [150]:
#ler_tabela("desp_por_fonte")

In [170]:
# Selecionando Recurso Ordinário - CFEM na Tabela # Pagina Inicial > Despesas > Fonte de Recurso > CFEM > Recurso Ordinário
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
try:
    # Localize o link pelo texto
    link_recurso_ord_cfem = driver.find_element(By.LINK_TEXT, "000.0911 - Recurso Ordinário - CFEM")
    
    # Clique no link
    link_recurso_ord_cfem.click()

except TimeoutException:
    print("Ocorreu um timeout ao esperar por um elemento.")
except NoSuchElementException:
    print("Não foi possível encontrar um elemento.")

### 2.1 Ler tabela e salvar links numa lista

In [152]:
#ler_tabela("naturezas_despesas")

In [171]:
desc_natureza_despesa:str = '4.4.90.51 - Obras e Instalações'
def seleciona_natureza_despesa(id_tabela, descrição_natureza):
  
  wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


  table = driver.find_element(By.ID, id_tabela)

  # links = set()   ---> garante que os links sejam unicos
  links = []  # Lista garante a ordem de inserção dos links
  rows = table.find_elements(By.TAG_NAME, 'tr')
  for row in rows:
    try:
        link = row.find_element(By.XPATH, './td[1]//a') # encontrar um link dentro da primeira coluna
        href = link.get_attribute('href')
        links.append(href)  # Add para set / append para list
    except:
        continue
    #for link in links:
    #print(link)

  link_cfem_obras = driver.find_element(By.LINK_TEXT, descrição_natureza)
  link_cfem_obras.click()

    
seleciona_natureza_despesa('tbTabela',desc_natureza_despesa)

In [154]:
#ler_tabela("credores_obras_e_instalacoes")

In [155]:
def ler_tabela_credores(nome_dados):
    global dados_tabelas

    # Aguarda até que o elemento da tabela esteja presente na página
    html_table = wait.until(EC.presence_of_element_located((By.ID, 'tbTabela')))
    
    # Passa o objeto html_table para o BeautifulSoup
    soup = BeautifulSoup(html_table.get_attribute('outerHTML'), 'html.parser')
    
    # Encontra o elemento da tabela
    table = soup.find('table')

    table = soup.find('table', {'id': 'tbTabela'})

    # Extraí os cabeçalhos
    headers = [header.get_text().strip() for header in table.find_all('th')]

    # Remove possíveis pontos nos headers
    headers_text = [header.replace('.', '') for header in headers]

    # Dicionário para armazenar os dados da tabela
    table_data = {}

    # Extrai os dados da tabela, somente das linhas cujo ID contenha "parLinha"
    for row in table.find_all('tr')[1:]:  # Começa a partir da segunda linha para pular o cabeçalho
        if "parLinha" in row.get('id', ''):  # Verifica se "parLinha" está no ID da linha
            cells = row.find_all('td')
            
            if len(cells) > 0:
                item = cells[0].get_text().strip().replace('.', '')  # Remove pontos do item
                # Remove todos os pontos dos dados de texto
                cells_text = [cell.get_text().strip().replace('.', '') for cell in cells]
                data = {
                    headers_text[i+1]: cells_text[i] for i in range(len(cells_text))
                }
                table_data[item] = data


    # Converte dados para JSON identado
    json_data = json.dumps(table_data, indent=4, ensure_ascii=False)
    
    # Lidando com diretório
    directory = 'json_output'
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{nome_dados}.json')
    
    # Salvando JSON em arquivo
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json_file.write(json_data)
    print(f'Dados salvos em: {file_path}')

    #driver.back()

ler_tabela_credores("credores")

Dados salvos em: json_output\credores.json


In [156]:
def ler_tabela_empenhos(nome_dados):
    global dados_tabelas

    # Aguarda até que o elemento da tabela esteja presente na página
    html_table = wait.until(EC.presence_of_element_located((By.ID, 'tbTabela')))
    
    # Passa o objeto html_table para o BeautifulSoup
    soup = BeautifulSoup(html_table.get_attribute('outerHTML'), 'html.parser')
    
    # Encontra o elemento da tabela
    table = soup.find('table')

    table = soup.find('table', {'id': 'tbTabela'})

    # Extraí os cabeçalhos
    headers = [header.get_text().strip() for header in table.find_all('th')]

    # Remove possíveis pontos nos headers
    headers_text = [header.replace('.', '') for header in headers]

    # Dicionário para armazenar os dados da tabela
    table_data = {}

    # Extrai os dados da tabela, somente das linhas cujo ID contenha "parLinha"
    for row in table.find_all('tr')[1:]:  # Começa a partir da segunda linha para pular o cabeçalho
        if "parLinha" in row.get('id', ''):  # Verifica se "parLinha" está no ID da linha
            cells = row.find_all('td')
            
            if len(cells) > 0:
                item = cells[1].get_text().strip().replace('.', '')  # Remove pontos do item
                # Remove todos os pontos dos dados de texto
                cells_text = [cell.get_text().strip().replace('.', '') for cell in cells]
                data = {
                    headers_text[i+1]: cells_text[i] for i in range(len(cells_text))
                }
                table_data[item] = data

    # Converte JSON
    json_data = json.dumps(table_data, indent=4, ensure_ascii=False)
    
    # Define the directory and file name
    directory = 'json_output'
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{nome_dados}.json')
    
    # Save the JSON data to a file
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json_file.write(json_data)
    print(f'Dados de empenho salvos em: {file_path}')

    #driver.back()

#ler_tabela_empenhos("teste_empenhos")

In [158]:
import json
import os
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Dicionário global para armazenar os dados
dados_tabelas = {}

# Função para ler e processar os dados das tabelas
def ler_descricao_empenho(nome_dados):
    global dados_tabelas

    try:
        # Coleta os dados da tabela tbHistorico
        html_historico = wait.until(EC.presence_of_element_located((By.ID, 'tbHistorico')))
        soup_historico = BeautifulSoup(html_historico.get_attribute('outerHTML'), 'html.parser')
        table_historico = soup_historico.find('table', {'id': 'tbHistorico'})

        headers_historico = [header.get_text().strip().replace('.', '') for header in table_historico.find_all('th')]

        historico_data = {}
        rows = table_historico.find_all('tr')
        
        # Pega apenas as duas últimas linhas da primeira coluna
        for row in rows[-2:]:
            cells = row.find_all('td')
            if len(cells) > 0:
                item = cells[0].get_text().strip().replace('.', '')
                if item not in historico_data:
                    cells_text = [cell.get_text().strip().replace('.', '') for cell in cells]

                for i in range(len(cells_text)):
                    if 'Credor:' in cells_text[i]:

                        cells_text[i] = cells_text[i].replace('Credor:', '').strip()
                        data = {
                        "credor: ": cells_text[0]
                        }

                    elif 'Nr Empenho' in cells_text[i]:

                        cells_text[i] = cells_text[i].replace('Nr Empenho', '').strip()
                        data = {
                            "Nr Empenho: ": cells_text[0]
                        }

                    historico_data[item] = data

        # Atualiza dados_tabelas com os dados coletados de tbHistorico
        if nome_dados not in dados_tabelas:
            dados_tabelas[nome_dados] = {'historico': historico_data}
        else:
            dados_tabelas[nome_dados]['historico'] = historico_data

    except Exception as e:
        print(f"Erro ao coletar dados de tbHistorico: {e}")

    try:
        # Coleta os dados da tabela tbTabela
        html_tabela = wait.until(EC.presence_of_element_located((By.ID, 'tbTabela')))
        soup_tabela = BeautifulSoup(html_tabela.get_attribute('outerHTML'), 'html.parser')
        table_tabela = soup_tabela.find('table', {'id': 'tbTabela'})

        # Inicializar a seção de descrições
        descricoes = {}

        # Iterar sobre os itens e suas informações
        for idx, row in enumerate(table_tabela.find_all('tr')[1:-1]):
            cells = row.find_all('td')
            if len(cells) > 0:
                item = cells[0].get_text().strip().replace('.', '')
                cells_text = [cell.get_text().strip().replace('.', '') for cell in cells]
                print(cells_text)

                # Construir o objeto de descrição
                if "CONTRATACAO DE OBRAS E SERVICOS." in cells_text[0]:
                    data = {
                        "Item": cells_text[0],
                        "Quantidade": cells_text[1],
                        "Unidade": cells_text[2],
                        "Valor Unitário": cells_text[3],
                        "Total": cells_text[4],
                    }
                    
                else:
                    item = "desc_completa"
                    data = {
                        "Item": cells_text[0],
                    }
                # Adicionar ao dicionário de descrições
                descricoes[item] = data

                if idx == 1:  # Segunda iteração
                    item = "desc_completa",
                    data = {
                        "Item": cells_text[0],
                    }  
                else:
                    data = {
                        "Item": cells_text[0],
                        "Quantidade": cells_text[1],
                        "Unidade": cells_text[2],
                        "Valor Unitário": cells_text[3],
                        "Total": cells_text[4],
                    }  

        # Atualiza dados_tabelas com histórico e descrições
        dados_tabelas[nome_dados]['descricoes'] = descricoes


    except Exception as e:
        print(f"Erro ao coletar dados de tbTabela: {e}")

    # Convert the data to a JSON string with indentation
    json_data = json.dumps(dados_tabelas, indent=4, ensure_ascii=False)
    
    # Define the directory and file name
    directory = 'json_output'
    if not os.path.exists(directory):
        os.makedirs(directory)
    file_path = os.path.join(directory, f'{nome_dados}.json')
    
    #Abrir o arquivo no modo de adição ('a' para append)
    with open(file_path, 'a', encoding='utf-8') as json_file:
        # Converter os dados JSON em uma string
        # Escrever os dados no arquivo
        json_file.write("," + json_data)  # Adicionar uma quebra de linha se necessário
    print(f'Dados de descrição adicionados em: {file_path}')



In [159]:
#ler_descricao_empenho("teste_desc_1")

In [160]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Dicionário para rastrear linhas processadas em cada nível
processed_rows = {0: set(), 1: set()}  

def encontrar_e_clicar_links(driver, tabela_id, depth=0):
    while True:
        # Atualizar o conteúdo da tabela
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.ID, tabela_id)))
        table_html = driver.find_element(By.ID, tabela_id).get_attribute('outerHTML')
        soup = BeautifulSoup(table_html, 'html.parser')
        rows = soup.find_all('tr')

        print(f"Depth {depth}: {rows}")

        new_links_found = False

        # Pular cabeçalho e linhas já processadas
        for index, row in enumerate(rows):
          if index < 2 or index in processed_rows[depth]:
              continue  

          processed_rows[depth].add(index)  # Marcar a linha como processada

          try:
              if depth < 2:
                link = row.find_all('td')[depth].find('a')
              if link is None:
                  continue
              link_href = link.get('href')  # Obter o atributo href do link

              # Use JavaScript to navegar para evitar StaleElementReferenceException
              driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH, f'//a[@href="{link_href}"]'))
              
              new_links_found = True

              wait.until(EC.presence_of_element_located((By.ID, tabela_id)))

              if depth < 1:
                  # Chama a função recursivamente para o próximo nível
                  encontrar_e_clicar_links(driver, tabela_id, depth+1)
                  
              else:
                  # Processa a descrição do empenho no último nível
                  ler_descricao_empenho(f"lista_todas_descricoes")
                  index=0

              driver.back()
              
              wait.until(EC.presence_of_element_located((By.ID, tabela_id)))
              # Atualizar a tabela após voltar
              table_html = driver.find_element(By.ID, tabela_id).get_attribute('outerHTML')
              soup = BeautifulSoup(table_html, 'html.parser')
              rows = soup.find_all('tr')
          except Exception as err:
              print(err)
              wait.until(EC.presence_of_element_located((By.ID, tabela_id)))
              table_html = driver.find_element(By.ID, tabela_id).get_attribute('outerHTML')
              soup = BeautifulSoup(table_html, 'html.parser')
              processed_rows[depth].remove(index)  # Se falhar, remover a linha dos processados para tentar novamente
              
          

        if not new_links_found:
            processed_rows[1] = set()
            break
            


# Chamar a função com a profundidade inicial 0
encontrar_e_clicar_links(driver, "tbTabela", 0)


Depth 0: [<tr><th align="center" class="Class_TituloCorFundo" colspan="7"><div style="text-align:center;">Credores</div><div style="float:right;margin-top:-16px;"><a href="javascript:MinMax('MinMaxtbTabela', 'tbTabela');"><img alt="Minimizar" id="MinMaxtbTabela" src="min.png"/></a></div></th></tr>, <tr class="Class_TituloRel"><th align="center">Nome</th><th align="center">CNPJ/CPF</th><th align="center">Valor Empenhado</th><th align="center">Valor Em Liquidação</th><th align="center">Valor Liquidado</th><th align="center">Valor Pago</th><th align="center">Valor Anulado</th></tr>, <tr bgcolor="#FFFFFF" class="Class_Relatorio" id="tbTabela_parLinha1" onmouseout="mudar_cor_out(this,'#FFFFFF');" onmouseover="mudar_cor_over(this);"><td align="left" style="white-space: nowrap;"><a href="javascript:IncluirHistorico('tbTabela_parLinha1');exibirAguarde();montaURLDetalhamentoItem('/pronimtb/index.asp?acao=3&amp;item=6&amp;visao=5&amp;ano=2024&amp;mesinicial=20240101&amp;mesfinal=20240131&amp;uni

# Formatando Dados

In [161]:
import json
from bson.decimal128 import Decimal128

# Função para converter valores monetários
def converter_valores(json_obj, nome_arquivo):
    for key, value in json_obj.items():
        if isinstance(value, dict):
            converter_valores(value, nome_arquivo)  # Chamar recursivamente com o mesmo nome do arquivo
        else:
            if isinstance(value, str) and value.startswith("R$"):
                # Remover "R$ " e converter para float
                valor_convertido = value.replace("R$ ", "").replace(".", "").replace(",", ".")
                # Atualizar valor no JSON como Decimal128
                json_obj[key] = str(Decimal128(valor_convertido))

### Despesas por Fonte

In [162]:
# Nome do arquivo
nome_arquivo = "desp_por_fonte.json"

# Ler o arquivo JSON
with open(f"json_output/{nome_arquivo}", 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Converter os valores no JSON, passando o nome do arquivo como parâmetro
converter_valores(json_data, nome_arquivo)

# Salvar o JSON atualizado de volta no arquivo
with open(f"json_output/{nome_arquivo}", 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

print("Arquivo atualizado com sucesso.")

Arquivo atualizado com sucesso.


### Natureza das Despesas

In [163]:
# Nome do arquivo
nome_arquivo = "naturezas_despesas.json"

# Ler o arquivo JSON
with open(f"json_output/{nome_arquivo}", 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Converter os valores no JSON, passando o nome do arquivo como parâmetro
converter_valores(json_data, nome_arquivo)

# Salvar o JSON atualizado de volta no arquivo
with open(f"json_output/{nome_arquivo}", 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

print("Arquivo atualizado com sucesso.")

Arquivo atualizado com sucesso.


### Credores referentes à Obras e Instalações

In [164]:
# Nome do arquivo
nome_arquivo = "credores_obras_e_instalacoes.json"

# Ler o arquivo JSON
with open(f"json_output/{nome_arquivo}", 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Converter os valores no JSON, passando o nome do arquivo como parâmetro
converter_valores(json_data, nome_arquivo)

# Salvar o JSON atualizado de volta no arquivo
with open(f"json_output/{nome_arquivo}", 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

print("Arquivo atualizado com sucesso.")

Arquivo atualizado com sucesso.


### Limpeza de Descrição (em desenvolvimento)

In [ ]:
# import json
# import os

# def limpar_json(nome_arquivo):
#     # Abrir o arquivo JSON e ler seu conteúdo
#     with open(nome_arquivo, 'r', encoding='utf-8') as arquivo:
#         conteudo = arquivo.read()
    
#     # Substituir "}{" por "},{"
#     conteudo_limpo = conteudo.replace('}{', '},{')

#     # Carregar o conteúdo limpo como um objeto JSON para validar
#     json_obj = json.loads(f'[{conteudo_limpo}]')
    
#     # Converter o objeto JSON de volta para uma string formatada
#     # Escrever o conteúdo limpo no novo arquivo
#     with open(f"{nome_arquivo}", 'w', encoding='utf-8') as file:
#         json.dump(json_obj, file, ensure_ascii=False, indent=4)
    
#     print(f'Arquivo limpo criado: {nome_arquivo}')

# # Exemplo de uso
# limpar_json('json_output/lista_todas_descricoes.json')


### Categorização (Em desenvolvimento)

In [ ]:
# import json

# def categorize_and_sum_values(json_file):
#     # Define categories and their associated keywords
#     categories = {
#         "pavimentação": ["pavimentação", "asfalto"],
#         "drenagem": ["drenagem", "pontes"],
#         "reformas": ["reforma", "reformas"],
#         "outros": []
#     }
    
#     # Load the JSON data from the file
#     with open(json_file, 'r', encoding='utf-8') as file:
#         data = json.load(file)
    
#     # Initialize totals dictionary
#     totals = {category: 0 for category in categories}
    
#     try:
#         # Access 'desc_completa' from the data
#         descricoes = data['teste_desc_1']['descricoes']['CONTRATACAO DE OBRAS E SERVICOS']
#         completa = data['teste_desc_1']['descricoes']['desc_completa']
        
#         # Extract item description and total expenditure
#         item_description = completa['Item'].lower()
#         item_total = descricoes['Total'].replace("R$ ", "").replace(".", "").replace(",", ".")
#         item_total = float(item_total)
        
#         # Determine the category for the item
#         categorized = False
#         for category, keywords in categories.items():
#             if any(keyword in item_description for keyword in keywords):
#                 totals[category] += item_total
#                 categorized = True
#                 break
#         if not categorized:
#             totals["outros"] += item_total
    
#         # Calculate the total overall expenditure
#         total_expenditure = sum(totals.values())
        
#         return total_expenditure, totals
    
#     except KeyError:
#         print("Erro: Estrutura do JSON fornecido não está conforme esperado. Verifique o formato.")
#         return None, None
#     except Exception as e:
#         print(f"Ocorreu um erro durante o processamento do JSON: {str(e)}")
#         return None, None

# # Example usage:
# total_expenditure, category_totals = categorize_and_sum_values("json_output/teste_desc_1.json")

# if total_expenditure is not None and category_totals is not None:
#     print(f"Total Expenditure: R$ {total_expenditure:.2f}")
#     for category, total in category_totals.items():
#         print(f"{category.capitalize()}: R$ {total:.2f}")


Total Expenditure: R$ 600000.00
Pavimentação: R$ 0.00
Drenagem: R$ 600000.00
Outros: R$ 0.00
